#

# World University Rankings

In [21]:
import pandas as pd
import requests

In [22]:
pd.set_option('display.max_rows', None)

## Countries

In [23]:
countries_url = "https://restcountries.com/v3.1/all"
countries_response = requests.get(countries_url)
countries : list[dict] = countries_response.json()
countries

[{'name': {'common': 'Christmas Island',
   'official': 'Territory of Christmas Island',
   'nativeName': {'eng': {'official': 'Territory of Christmas Island',
     'common': 'Christmas Island'}}},
  'tld': ['.cx'],
  'cca2': 'CX',
  'ccn3': '162',
  'cca3': 'CXR',
  'independent': False,
  'status': 'officially-assigned',
  'unMember': False,
  'currencies': {'AUD': {'name': 'Australian dollar', 'symbol': '$'}},
  'idd': {'root': '+6', 'suffixes': ['1']},
  'capital': ['Flying Fish Cove'],
  'altSpellings': ['CX', 'Territory of Christmas Island'],
  'region': 'Oceania',
  'subregion': 'Australia and New Zealand',
  'languages': {'eng': 'English'},
  'translations': {'ara': {'official': 'جزيرة كريسماس',
    'common': 'جزيرة كريسماس'},
   'bre': {'official': 'Tiriad Enez Christmas', 'common': 'Enez Christmas'},
   'ces': {'official': 'Teritorium Vánočního ostrova',
    'common': 'Vánoční ostrov'},
   'cym': {'official': 'Tiriogaeth yr Ynys y Nadolig',
    'common': 'Ynys y Nadolig'},
  

In [24]:
countries_dict : dict = {}
for country_item in countries:
    name : dict = country_item.get("name", {})
    common_name : str = name.get("common", "")
    official_name : str = name.get("official", "")
    countries_dict[official_name] = common_name
    alt_spellings : list[str] = country_item.get("altSpellings", [])
    for alt_spelling in alt_spellings:
        countries_dict[alt_spelling] = common_name
countries_dict

{'Territory of Christmas Island': 'Christmas Island',
 'CX': 'Christmas Island',
 'State of Eritrea': 'Eritrea',
 'ER': 'Eritrea',
 'ሃገረ ኤርትራ': 'Eritrea',
 'Dawlat Iritriyá': 'Eritrea',
 'ʾErtrā': 'Eritrea',
 'Iritriyā': 'Eritrea',
 'Independent State of Samoa': 'Samoa',
 'WS': 'Samoa',
 'Malo Saʻoloto Tutoʻatasi o Sāmoa': 'Samoa',
 'Republic of North Macedonia': 'North Macedonia',
 'MK': 'North Macedonia',
 'The former Yugoslav Republic of Macedonia': 'North Macedonia',
 'Macedonia, The Former Yugoslav Republic of': 'North Macedonia',
 'Република Северна Македонија': 'North Macedonia',
 'Republic of Djibouti': 'Djibouti',
 'DJ': 'Djibouti',
 'Jabuuti': 'Djibouti',
 'Gabuuti': 'Djibouti',
 'République de Djibouti': 'Djibouti',
 'Gabuutih Ummuuno': 'Djibouti',
 'Jamhuuriyadda Jabuuti': 'Djibouti',
 'Hashemite Kingdom of Jordan': 'Jordan',
 'JO': 'Jordan',
 'al-Mamlakah al-Urdunīyah al-Hāshimīyah': 'Jordan',
 'Islamic Republic of Pakistan': 'Pakistan',
 'PK': 'Pakistan',
 'Pākistān': 'Pa

In [25]:
def trim_bracket(university : str) -> str:
    start = university.find("(")
    end = university.find(")")
    if start >= end:
        return university
    return university[:start].strip()

In [26]:
def map_university(university : str) -> str:
    university_dict = {
        "EPFL": "École Polytechnique Fédérale de Lausanne",
        "KAIST - Korea Advanced Institute of Science & Technology": "Korea Advanced Institute of Science and Technology",
        "Ludwig-Maximilians-Universität München": "University of Munich",
        "Nanyang Technological University, Singapore": "Nanyang Technological University",
        "University of Minnesota Twin Cities": "University of Minnesota, Twin Cities",
        "Trinity College Dublin, The University of Dublin": "Trinity College Dublin",
        "Purdue University": "Purdue University - West Lafayette",
        "University of Wisconsin Milwaukee": "University of Wisconsin-Milwaukee",
        "UCL": "University College London",
        "Université PSL": "Paris Sciences et Lettres – PSL Research University Paris",
        "University of San Francisco": "University of California, San Francisco",
    }
    if university in university_dict:
        return university_dict[university]
    start_the = university.find("The ")
    if start_the == 0:
        return university.replace("The ", "")
    return university.replace('’', "'").replace(" And ", " and ")

In [27]:
qs_url = "https://www.topuniversities.com/rankings/endpoint?nid=3897789&page=0&items_per_page=10000"
qs_response = requests.get(qs_url)
qs_data : dict = qs_response.json()
score_nodes : list[dict] = qs_data.get("score_nodes")
qs_rankings = []
for score_node in score_nodes:
    rank : float = float(score_node.get("rank", "-1"))
    university : str = trim_bracket(score_node.get("title", "").strip())
    university : str = map_university(university)
    country_value : str | bool = score_node.get("country", "")
    country_name = trim_bracket(country_value if type(country_value) != bool else "")
    country = country_name if country_name not in countries_dict else countries_dict[country_name]
    country = country.replace("SAR", "").strip()
    qs_rankings.append({
        "university": university,
        "country": country,
        "rank_qs": rank
    })
qs_rankings_data_frame : pd.DataFrame = pd.DataFrame(qs_rankings)
qs_rankings_data_frame.to_csv("./csv/qs.csv", index=False)
qs_universities : list[str] = qs_rankings_data_frame['university'].unique()
qs_countries : list[str] = qs_rankings_data_frame['country'].unique()
qs_rankings_data_frame

,university,country,rank_qs
0,Massachusetts Institute of Technology,United States,1.0
1,University of Cambridge,United Kingdom,2.0
2,University of Oxford,United Kingdom,3.0
3,Harvard University,United States,4.0
4,Stanford University,United States,5.0
5,Imperial College London,United Kingdom,6.0
6,ETH Zurich,Switzerland,7.0
7,National University of Singapore,Singapore,8.0
8,University College London,United Kingdom,9.0
9,"University of California, Berkeley",United States,10.0


In [28]:
vietnam_qs_rankings_data_frame = qs_rankings_data_frame[qs_rankings_data_frame['country'] == 'Vietnam']
vietnam_qs_rankings_data_frame

,university,country,rank_qs
516,Duy Tan University,Vietnam,517.0
725,Ton Duc Thang University,Vietnam,726.0
1001,Viet Nam National University Ho Chi Minh City,Vietnam,1002.0
1002,"Vietnam National University, Hanoi",Vietnam,1003.0
1240,Hanoi University of Science and Technology,Vietnam,1241.0


In [29]:
australia_qs_rankings_data_frame = qs_rankings_data_frame[qs_rankings_data_frame['country'] == 'Australia']
australia_qs_rankings_data_frame.reset_index()

,index,university,country,rank_qs
0,13,University of Melbourne,Australia,14.0
1,18,University of New South Wales,Australia,19.0
2,19,University of Sydney,Australia,20.0
3,33,Australian National University,Australia,34.0
4,41,Monash University,Australia,42.0
5,42,University of Queensland,Australia,43.0
6,71,University of Western Australia,Australia,72.0
7,88,University of Adelaide,Australia,89.0
8,89,University of Technology Sydney,Australia,90.0
9,130,Macquarie University,Australia,131.0


In [30]:
united_kingdom_qs_rankings_data_frame = qs_rankings_data_frame[qs_rankings_data_frame['country'] == 'United Kingdom']
united_kingdom_qs_rankings_data_frame.reset_index()

,index,university,country,rank_qs
0,1,University of Cambridge,United Kingdom,2.0
1,2,University of Oxford,United Kingdom,3.0
2,5,Imperial College London,United Kingdom,6.0
3,8,University College London,United Kingdom,9.0
4,21,University of Edinburgh,United Kingdom,22.0
5,31,University of Manchester,United Kingdom,32.0
6,39,King's College London,United Kingdom,40.0
7,44,London School of Economics and Political Science,United Kingdom,45.0
8,54,University of Bristol,United Kingdom,55.0
9,66,University of Warwick,United Kingdom,67.0


In [31]:
united_states_qs_rankings_data_frame = qs_rankings_data_frame[qs_rankings_data_frame['country'] == 'United States']
united_states_qs_rankings_data_frame.reset_index()

,index,university,country,rank_qs
0,0,Massachusetts Institute of Technology,United States,1.0
1,3,Harvard University,United States,4.0
2,4,Stanford University,United States,5.0
3,9,"University of California, Berkeley",United States,10.0
4,10,University of Chicago,United States,11.0
5,11,University of Pennsylvania,United States,12.0
6,12,Cornell University,United States,13.0
7,14,California Institute of Technology,United States,15.0
8,15,Yale University,United States,16.0
9,17,Princeton University,United States,18.0
